# 태풍 솔릭(Soulik) 모의를 위한 ROMS 영역의<br/>HYCOM L.B.Cs 자료 가져오기 by NetcdfSubset

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
from datetime import datetime, timedelta
from urllib import parse, request
from multiprocessing import Process

#import numpy as np
from netCDF4 import Dataset, num2date, date2num

## 1. 정보 설정

In [3]:
# 실험명
expname = 'Soulik'; grd_version = 'v4'

# 경계장 날짜 설정
date_beg = datetime(2018,8,21,0) # year, month, day, hour
date_end = datetime(2018,8,25,0) # year, month, day, hour

server = 'https://ncss.hycom.org/thredds/ncss'
version = 'GLBv0.08'; expt = 'expt_93.0'
server = '/'.join([server,version,expt])

strideTime  = 1 # time resolution. ex) every 3hours * 1 = 3hours
#strideVert  = 2 # vertical resolution. ex) 40level / 2 = 20level
strideHoriz = 2 # horizontal resolution. ex) 1/12deg * 2 = 1/6deg
extraHoriz  = 2.0 # in degrees. use 2 times of delta X or delta Y.

# ROMS 격자 정보 가져오기
roms_grid_fname = '../Grid/'+expname+'_grd_'+grd_version+'.nc'
if not os.path.isfile(roms_grid_fname):
    print("==>Error: not found - %s"%roms_grid_fname)
else:
    with Dataset(roms_grid_fname) as roms_grid:
        minmaxLonR = [round(roms_grid['lon_vert'][:].min()-extraHoriz,3), # west
                      round(roms_grid['lon_vert'][:].max()+extraHoriz,3)] # east
        minmaxLatR = [round(roms_grid['lat_vert'][:].min()-extraHoriz,3), # south
                      round(roms_grid['lat_vert'][:].max()+extraHoriz,3)] # north
        print('==>Info: ', minmaxLonR, minmaxLatR)
        
# 기본 url 생성 (시간 정보 제외)
#https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0
#       ?var=surf_el&var=salinity&var=water_temp&var=water_u&var=water_v
#       &north=53&west=116&east=161&south=19
#       &disableProjSubset=on&horizStride=6
#       &time_start=2018-08-18T12%3A00%3A00Z&time_end=2018-08-24T12%3A00%3A00Z&timeStride=1
#       &vertCoord=&addLatLon=true&accept=netcdf4
qGrid = [('north',minmaxLatR[1]),('west',minmaxLonR[0]),('east',minmaxLonR[1]),('south',minmaxLatR[0]), # NWES
         ('disableProjSubset','on'),('horizStride',strideHoriz),
         ('vertCoord',''),('addLatLon','true'),('accept','netcdf4')]
#qTime = [('time_start',ymdh_beg),('time_end',ymdh_end),('timeStride',strideTime)] # time start, end, stride

params = parse.urlencode(qGrid, encoding='UTF-8', doseq=True)
urls = {'ts3z': "%s/ts3z?%s&var=water_temp&var=salinity"%(server,params),
        'uv3z': "%s/uv3z?%s&var=water_u&var=water_v"%(server,params),
        'ssh' : "%s/ssh?%s&var=surf_el"%(server,params)}; print(urls)

==>Info:  [96.279, 192.521] [3.162, 63.558]
{'ts3z': 'https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/ts3z?north=63.558&west=96.279&east=192.521&south=3.162&disableProjSubset=on&horizStride=2&vertCoord=&addLatLon=true&accept=netcdf4&var=water_temp&var=salinity', 'uv3z': 'https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/uv3z?north=63.558&west=96.279&east=192.521&south=3.162&disableProjSubset=on&horizStride=2&vertCoord=&addLatLon=true&accept=netcdf4&var=water_u&var=water_v', 'ssh': 'https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/ssh?north=63.558&west=96.279&east=192.521&south=3.162&disableProjSubset=on&horizStride=2&vertCoord=&addLatLon=true&accept=netcdf4&var=surf_el'}


## 2. HYCOM으로부터 수온, 염분, U, V, SSH 자료 가져오기

> **Note**) HYCOM 서버의 상황을 고려하여 시간 단위로 자료를 받아야 하며,<br/>
>      시간을 절약하기 위하여 각 변수마다 병렬로 다운로드를 동시에 수행함.

In [6]:
# Note) HYCOM 자료는 서버와의 반응이 느리면 (보통 5분 제한) 끊기는 현상이 발생하므로 
#       시간 단위로 받은 후 합치는 방법을 사용할 것을 추천함

# 자료 저장 범용 함수 정의
def getHYCOM(prefix, tinfo, dt, vnames, clobber=False):
    outfile = 'HYCOM/HYCOM4%s_%s_%s.nc4'%(expname,prefix,dt.strftime('%Y-%m-%dT%H:%M:%SZ'))
    if os.path.isfile(outfile):
        if clobber: os.remove(outfile)
        else: print('==>Info: already exists %s'%outfile); return

    url = "%s&%s"%(urls[prefix],tinfo)
    print('==>Info[%s]: %s to %s'%(prefix,url,outfile))
    _=request.urlretrieve(url, outfile)
    
    # Note) HYCOM의 기본 _FillValue(-30000s)를 pyroms에서 인식하지 못하기 때문에 nc를 unpacking하여 NaN으로 바꾸어 줌.
    for v in vnames:
        os.system('ncpdq -O -P upk %s %s'%(outfile,outfile))
        #os.system('ncatted -a _FillValue,%s,o,d,NaN %s'%(v,outfile))
        #os.system('ncatted -a missing_value,%s,o,d,NaN %s'%(v,outfile))
        
        
if not os.path.isdir('HYCOM'): os.makedirs('HYCOM')
    
dt = date_beg
while(dt <= date_end):
    query=[('time_start',dt.strftime('%Y-%m-%dT%H:%M:%SZ')), # start time
           ('time_end',  dt.strftime('%Y-%m-%dT%H:%M:%SZ')), # end time
           ('timeStride',1)]                              # stride
    tinfo = parse.urlencode(query, encoding='UTF-8', doseq=True)
    
    pTS = Process(target=getHYCOM, args=('ts3z',tinfo,dt,['water_temp','salinity'],False)); pTS.start() # temp, salt
    pUV = Process(target=getHYCOM, args=('uv3z',tinfo,dt,['water_u','water_v'],False)); pUV.start() # u, v
    pSSH= Process(target=getHYCOM, args=('ssh', tinfo,dt,['surf_el'],False)); pSSH.start()# ssh
    pTS.join(); pUV.join(); pSSH.join() 
    dt += timedelta(hours=3*strideTime)

==>Info[ts3z]: https://ncss.hycom.org/thredds/ncss/GLBv0.08/expt_93.0/ts3z?north=63.558&west=96.279&east=192.521&south=3.162&disableProjSubset=on&horizStride=2&vertCoord=&addLatLon=true&accept=netcdf4&var=water_temp&var=salinity&time_start=2018-08-21T00%3A00%3A00Z&time_end=2018-08-21T00%3A00%3A00Z&timeStride=1 to HYCOM/HYCOM4Soulik_ts3z_2018-08-21T00:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_uv3z_2018-08-21T00:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ssh_2018-08-21T00:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ts3z_2018-08-21T03:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_uv3z_2018-08-21T03:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ssh_2018-08-21T03:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ts3z_2018-08-21T06:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_uv3z_2018-08-21T06:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ssh_2018-08-21T06:00:00Z.nc4
==>Info: already exists HYCOM/HYCOM4Soulik_ts3z_2018-08